In [1]:
import sys
sys.path.append("/Users/PRVATE/Documents/tf_transformers/src/")

In [5]:
from transformers import TFGPT2Model
from tf_transformers.models import GPT2Encoder

import tensorflow as tf
import json

from tf_transformers.utils import convert_gpt2_hf_to_tf_transformers
import os

In [3]:
# Load HF model

# Always do this
tf.keras.backend.clear_session()

model_hf_location = '/Users/PRVATE/HUggingFace_Models/gpt2/'
model_hf = TFGPT2Model.from_pretrained(model_hf_location)

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at /Users/PRVATE/HUggingFace_Models/gpt2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [6]:
# Load tf_transformers model
# Most config we will be providing

# Default configs for the model
# Default configs for the model
model_config_dir = '/Users/PRVATE/Documents/tf_transformers/model_configs/'
model_name = 'gpt2_base'
config_location = os.path.join(model_config_dir, model_name, 'config.json')
config = json.load(open(config_location))
# Always do this
tf.keras.backend.clear_session()

# tf_transformers Layer (an extension of Keras Layer)
# This is not Keras model, but extension of keras Layer

model_layer = GPT2Encoder(config=config,
                      name='gpt2',
                      mask_mode=config['mask_mode'],
                      is_training=False, 
                      use_dropout=False,
                      )
# model_dir = None, because we have not initialized the model with proper variable values
model_tf_transformers = model_layer.get_and_load_model(model_dir=None)

INFO:absl:We are overwriding `is_training` is False to `is_training`                     to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids_1:0", shape=(None, None), dtype=int32)


In [7]:
convert_gpt2_hf_to_tf_transformers(model_hf, model_tf_transformers, config)

INFO:absl:Deleteing huggingface model for saving memory
INFO:absl:Done assigning variables weights . Total 100


In [8]:
# Please have a look at tf_transformers/extra/*.py for reference values

input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])

input_mask = tf.ones_like(input_ids)
input_type_ids = tf.zeros_like(input_ids)

inputs = {'input_ids': input_ids, 
}

results_tf_transformers   = model_tf_transformers(inputs)
for k, r in results_tf_transformers.items():
    if isinstance(r, list):
        continue
    print(k, '-->', tf.reduce_sum(r), '-->', r.shape)
    
    
# For GPT2 Base 

# token_embeddings --> tf.Tensor(2371.2751, shape=(), dtype=float32) --> (2, 5, 768)
# token_logits --> tf.Tensor(-34781260.0, shape=(), dtype=float32) --> (2, 5, 50257)
# last_token_logits --> tf.Tensor(-8346980.5, shape=(), dtype=float32) --> (2, 50257)

token_embeddings --> tf.Tensor(2371.2754, shape=(), dtype=float32) --> (2, 5, 768)
token_logits --> tf.Tensor(-34781264.0, shape=(), dtype=float32) --> (2, 5, 50257)
last_token_logits --> tf.Tensor(-8346981.0, shape=(), dtype=float32) --> (2, 50257)


In [9]:
# Huggingface Model
input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])

results_hf = model_hf([input_ids])
for k in results_hf:
    if isinstance(k, tf.Tensor):
        print(tf.reduce_sum(k), '-->', k.shape)

tf.Tensor(2370.4778, shape=(), dtype=float32) --> (2, 5, 768)


In [11]:
model_name = 'gpt2'
model_loc  = '/Users/PRVATE/tf_transformers_models/'
checkpoint_dir = os.path.join(model_loc, model_name)
if os.path.exists(checkpoint_dir):
    raise FileExistsError()

# If you want to save the model as checkpoints
checkpoint = tf.train.Checkpoint(model=model_tf_transformers)
manager = tf.train.CheckpointManager(
    checkpoint, directory=checkpoint_dir, max_to_keep=1
)
manager.save()
print("Saved at {}".format(manager.latest_checkpoint))

Saved at /Users/PRVATE/tf_transformers_models/gpt2/ckpt-1
